In [1]:
 !pip install rectools

  Obtaining dependency information for rectools from https://files.pythonhosted.org/packages/f2/2c/548002983de17fba745d88eded49bf67007ef54db35272e3cf9605de50fb/rectools-0.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for implicit<0.8.0,>=0.7.1 from https://files.pythonhosted.org/packages/cd/cc/deac70cae8cc32c9885d0cd73bc66e1b3cbea36ae7080b8c83995eaf5322/implicit-0.7.2-cp310-cp310-manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 68.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 82.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [2]:
!pip install dill

In [3]:
import pickle
from pathlib import Path
import dill

import numpy as np
import pandas as pd
import requests
from tqdm.auto import tqdm
from typing import Dict
from scipy.stats import mode
from pprint import pprint
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender
import warnings

from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, MeanInvUserFreq, MRR, calc_metrics
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models.popular import PopularModel
from rectools.model_selection import TimeRangeSplitter


import scipy as sp

from collections import Counter

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [5]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [6]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()


In [7]:
interactions_df = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

interactions_df.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

In [8]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [9]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман, Дэн Риззуто, Кендес Илэйн Калтраро, Кит Джардин, Лекса Дойг, Майкл Джей Уайт, Майкл Шэнкс, Майкл Эклунд, Питер Брайант, Питер Кент, Стив Бачич, Стив ...","Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказа...","Тактическая, сила, 2011, Канада, бандиты, гангстеры, преступления, преодоление, трудностей, убийства, убийцы, настоящие, мужчины, риск, недоразумение, силы, правопорядка, борьба, за, выживание, сп..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джеймс, Долли Уэллс, Дэвид Сибли, Камилла Юкан, Кевин Матадин, Люси Темпл, Макс Радд, Мартин Эткинсон, Мишель Финч, Питер Дин Джексон, Пол Голдсмит, Ричард...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джеймс и Долли Уэллс в драме Эндрю Хэйя. Супруги Кейт и Джефф Мерсеры готовятся к важной дате – с того дня, как они поженились, прошло целых сорок пять л...","45, лет, 2015, Великобритания, брак, жизнь, любовь, психология, семейные, проблемы, семья, отношения, конфликты, отношениях, отношения, паре, отношения, мужчины, женщины, семейный, конфликт, смерть"
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алексей Васильев, Анатолий Папанов, Андрей Данилов, Борис Зайденберг, Борис Коковкин, Виктор Карасев, Галина Беляева, Е. Рахманова, Елена Аржаник, Ирина Са...","Расчетливая чаровница из советского кинохита «Мой ласковый и нежный зверь», актриса-дебютантка Галина Беляева снова становится неопытным и робким подростком в спортивном фильме «Все решает мгнове...","Все, решает, мгновение, 1978, СССР, сильные, женщины, спорт, преодоление, трудностей, мечта, соперничество, подростки, женщины, успех, коллектив, команда, фобии, спортсмены, отношения, между, трен..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Виитала, Сусанна Хаависто, Кари Кетонен, 

In [10]:
users.user_id.max()

1097558

In [11]:
items.item_id.max()

16518

In [12]:
interactions_df

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


# Обработка холодных юзеров


придумать, что делать с холодными пользователями в тесте. Сделайте рекомендации для них (обратите внимание на rectools.models.popular) (3 балла)

реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы (3 балла)

Для успешной обработки популярного за последнее время мы воспользуемся разбиением наших данных по году выпуска и жанрам. Это будут наши фичи

Мы возьмем самую простую модель PopularModel из rectools

In [13]:
_, bins = pd.qcut(items['release_year'], 10, retbins=True)
labels = bins[:-1]

items_for_years = pd.DataFrame(
    {
        'id': items['item_id'],
        'value': pd.cut(items['release_year'], bins=bins, labels=bins[:-1]),
        'feature': 'release_year'
    }
)

In [14]:
items['genre'] = items['genres'].str.split(',')
items[['genre', 'genres']].head(3)

,genre,genres
0,"[драмы, зарубежные, детективы, мелодрамы]","драмы, зарубежные, детективы, мелодрамы"
1,"[зарубежные, приключения, комедии]","зарубежные, приключения, комедии"
2,"[криминал, зарубежные, триллеры, боевики, комедии]","криминал, зарубежные, триллеры, боевики, комедии"


In [15]:
items_for_genre = items[['item_id', 'genre']].explode('genre')
items_for_genre.columns = ['id', 'value']
items_for_genre['feature'] = 'genre'
items_for_genre.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


Объединение и фильтрация признаков так, чтобы они оставались только для элементов, с которыми взаимодействовали юзеры.

In [16]:
items_year_genre = pd.concat([items_for_genre, items_for_years])
items_year_genre = items_year_genre[items_year_genre['id'].isin(interactions_df['item_id'])]

In [17]:
items_for_genre.head().tail()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [18]:
dataset = Dataset.construct(
    interactions_df=interactions_df,
    user_features_df=None,
    item_features_df=items_year_genre,
    cat_item_features=['genre', 'release_year']
)

In [19]:
popular_model = PopularModel()
popular_model.fit(dataset)

Рекомендуем и совмещаем, чтобы нам выдались названия того, что порекомендовали

In [20]:
# join titles from items
popular_recommendations = popular_model.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=20,
    filter_viewed=False
).merge(items[['item_id', 'title']],
       on='item_id',
       how='left')

In [21]:
popular_recommendations.head(10)

,user_id,item_id,score,rank,title
0,176549,10440,202457.0,1,Хрустальный
1,176549,15297,193123.0,2,Клиника счастья
2,176549,9728,132865.0,3,Гнев человеческий
3,176549,13865,122119.0,4,Девятаев
4,176549,4151,91167.0,5,Секреты семейной жизни
5,176549,3734,74803.0,6,Прабабушка легкого поведения
6,176549,2657,68581.0,7,Подслушано
7,176549,4880,55043.0,8,Афера
8,176549,142,45367.0,9,Маша
9,176549,6809,40372.0,10,Дуров


Это все мы сделали, чтобы получить веса модели для работы сервиса


In [22]:
models_dir = Path('/kaggle/working/models')
models_dir.mkdir(exist_ok=True)

In [23]:
with open(f'{models_dir}/popular.dill', 'wb') as save_file:
    dill.dump(popular_model, save_file)

# Обучение и тест модели

Этот код для того чтобы получить отфильтрованные айдишники всего периода обучения для трейна, чтобы модель из сервиса работала онлайн

In [24]:
N_SPLITS = 1
TEST_SIZE = '49D'

In [25]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [26]:
interactions = Interactions(interactions_df)
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-07-05 00:00:00', freq='49D'),
  Timestamp('2021-08-23 00:00:00', freq='49D'))]

In [27]:
fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()



==================== Fold 0
{'end': Timestamp('2021-08-23 00:00:00', freq='49D'),
 'i_split': 0,
 'start': Timestamp('2021-07-05 00:00:00', freq='49D'),
 'test': 1043653,
 'test_items': 9072,
 'test_users': 197624,
 'train': 2939612,
 'train_items': 14397,
 'train_users': 603297}


In [28]:
with open(f'{models_dir}/train_ids.npy', 'wb') as f:
    a = np.save(f, np.asarray(df_train.index))

Теперь разобьем на 7 фолдов для обучения как и было на практике

In [29]:
N_SPLITS = 7
TEST_SIZE = '7D'

In [30]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [31]:
interactions = Interactions(interactions_df)
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-07-05 00:00:00', freq='7D'),
  Timestamp('2021-07-12 00:00:00', freq='7D')),
 (Timestamp('2021-07-12 00:00:00', freq='7D'),
  Timestamp('2021-07-19 00:00:00', freq='7D')),
 (Timestamp('2021-07-19 00:00:00', freq='7D'),
  Timestamp('2021-07-26 00:00:00', freq='7D')),
 (Timestamp('2021-07-26 00:00:00', freq='7D'),
  Timestamp('2021-08-02 00:00:00', freq='7D')),
 (Timestamp('2021-08-02 00:00:00', freq='7D'),
  Timestamp('2021-08-09 00:00:00', freq='7D')),
 (Timestamp('2021-08-09 00:00:00', freq='7D'),
  Timestamp('2021-08-16 00:00:00', freq='7D')),
 (Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

Для тюнинга добавила еще BM25 distance и метрики как были в прошлом дз, но только для К=10

***провести эксперименты с параметрами оффлайн валидации и сделать выводы (3 балла)***

***реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы (3 балла)*** - Я выбрала типы кнн моделей

In [32]:
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),

    "map@10": MAP(k=10),
    "mrr@10": MRR(k=10),

    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@10": Serendipity(k=10),
}

models = {
    'cosine_userknn': CosineRecommender(),
    'tfidf_userknn': TFIDFRecommender(),
    'bm25_userknn': BM25Recommender()

}

Так как у меня не прошел тест предложенный на практике userknn, то тут я уже покажу как исправила ноутбук и переписала весь класс

In [79]:
class UserKnn:
    """Class for fit-perdict UserKNN model
    """

    def __init__(self, model, N_users: int = 50):

        self.model = model

        self.dataset = interactions_df
        self.n = self.dataset.shape[0]
        self.idf, self.idf_dict = self.prepare_idf()

        # это чтобы удобнее в сервисе сразу подгружать датасет c
        # отфильтрованными холодными юзерами и айтемами
        train_ids = np.load("/kaggle/working/models/train_ids.npy",
                            allow_pickle=True)
        train = self.dataset.loc[train_ids]
        self.dataset = train

        self.watched, self.watched_dict = self.prepare_watched()

        self.get_mappings(train)

        self.mapper = self._generate_recs_mapper(N=N_users)

    def recommend(self, test: pd.DataFrame, N_recs: int = 10):

        mapper = self._generate_recs_mapper(
            N=50
        )

        recs = pd.DataFrame({'user_id': test['user_id'].unique()})
        recs['sim_user_id'], recs['sim'] = zip(*recs['user_id'].map(mapper))
        recs = recs.set_index('user_id').apply(pd.Series.explode).reset_index()

        recs = recs[~(recs['user_id'] == recs['sim_user_id'])]\
            .merge(self.watched, on=['sim_user_id'], how='left')\
            .explode('item_id')\
            .sort_values(['user_id', 'sim'], ascending=False)\
            .drop_duplicates(['user_id', 'item_id'], keep='first')\
            .merge(self.idf, left_on='item_id', right_on='index', how='left')

        recs['score'] = recs['sim'] * recs['idf']
        recs = recs.sort_values(['user_id', 'score'], ascending=False)
        recs['rank'] = recs.groupby('user_id').cumcount() + 1
        return recs[recs['rank'] <= N_recs][['user_id', 'item_id', 'score', 'rank']]

        return predictions

    def get_mappings(self, train):
        self.users_inv_mapping = dict(enumerate(train['user_id'].unique()))
        self.users_mapping = {v: k for k, v in self.users_inv_mapping.items()}

        self.items_inv_mapping = dict(enumerate(train['item_id'].unique()))
        self.items_mapping = {v: k for k, v in self.items_inv_mapping.items()}

    def _generate_recs_mapper(self, N: int):
        def _recs_mapper(user):
            user_id = self.users_mapping[user]
            recs = self.model.similar_items(user_id, N=N)
            maps = [self.users_inv_mapping[user] for user in recs[0]], \
                list(sim for sim in recs[1])
            maps[0].pop(0)
            maps[1].pop(0)
            return maps

        return _recs_mapper

    def get_matrix(self, df: pd.DataFrame,
                   user_col: str = 'user_id',
                   item_col: str = 'item_id',
                   weight_col: str = None,
                   users_mapping: Dict[int, int] = None,
                   items_mapping: Dict[int, int] = None):

        if weight_col:
            weights = df[weight_col].astype(np.float32)
        else:
            weights = np.ones(len(df), dtype=np.float32)

        self.interaction_matrix = sp.sparse.coo_matrix((
            weights,
            (
                df[item_col].map(self.items_mapping.get),
                df[user_col].map(self.users_mapping.get)
            )
            ))

        self.watched = df\
            .groupby(user_col, as_index=False)\
            .agg({item_col: list})\
            .rename(columns={user_col: 'sim_user_id'})

        return self.interaction_matrix

    def fit(self,train):
        self.user_knn = self.model
        self.get_mappings(train)
        self.weights_matrix = self.get_matrix(train,
                                              users_mapping=self.users_mapping,
                                              items_mapping=self.items_mapping)

        self.user_knn.fit(self.weights_matrix)


    def prepare_watched(self):
        watched = self.dataset.groupby('user_id').agg({'item_id': list})

        watched_dict = {user_id: items["item_id"] for user_id, items
                        in watched.iterrows()}
        return watched, watched_dict

    def get_watched_dict(self):
        return self.watched_dict

    def prepare_idf(self):
        cnt = Counter(self.dataset['item_id'].values)
        idf = pd.DataFrame.from_dict(cnt, orient='index',
                                     columns=['doc_freq']).reset_index()

        idf['idf'] = idf['doc_freq'].apply(lambda x:
                                           np.log((1 +  self.n) / (1 + x) + 1))

        idf_dict = {items["index"]: items["idf"] for ind, items in
                    idf.iterrows()}
        return idf, idf_dict


In [39]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)

        recos = userknn_model.recommend(df_test)

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )

        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)
        


==================== Fold 0
{'end': Timestamp('2021-07-12 00:00:00', freq='7D'),
 'i_split': 0,
 'start': Timestamp('2021-07-05 00:00:00', freq='7D'),
 'test': 204269,
 'test_items': 6199,
 'test_users': 83649,
 'train': 2939612,
 'train_items': 14397,
 'train_users': 603297}


  0%|          | 0/603297 [00:00<?, ?it/s]

  0%|          | 0/603297 [00:00<?, ?it/s]

  0%|          | 0/603297 [00:00<?, ?it/s]


==================== Fold 1
{'end': Timestamp('2021-07-19 00:00:00', freq='7D'),
 'i_split': 1,
 'start': Timestamp('2021-07-12 00:00:00', freq='7D'),
 'test': 216010,
 'test_items': 6314,
 'test_users': 84668,
 'train': 3239125,
 'train_items': 14730,
 'train_users': 646423}


...


==================== Fold 6
{'end': Timestamp('2021-08-23 00:00:00', freq='7D'),
 'i_split': 6,
 'start': Timestamp('2021-08-16 00:00:00', freq='7D'),
 'test': 298878,
 'test_items': 6679,
 'test_users': 110076,
 'train': 5051815,
 'train_items': 15577,
 'train_users': 906071}


  0%|          | 0/906071 [00:00<?, ?it/s]

  0%|          | 0/906071 [00:00<?, ?it/s]

  0%|          | 0/906071 [00:00<?, ?it/s]

CPU times: user 7h 14min 13s, sys: 1min 18s, total: 7h 15min 32s
Wall time: 4h 53s


In [41]:
for model_name in tqdm(models.keys()):
    with open(f'{model_name}.dill', 'wb') as save_file:
        dill.dump(models[model_name] , save_file)

  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
# Aggregate metrics by folds and compare models
pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset,color='lightcoral', axis=0)
    .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
)

In [62]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,mrr@10,map@10,novelty@10,serendipity@10
0,0,cosine_userknn,0.005528,0.031079,0.009066,0.005315,7.087192,0.000038
1,0,tfidf_userknn,0.008790,0.051680,0.014771,0.009536,7.184546,0.000038
2,0,bm25_userknn,0.003364,0.018180,0.005604,0.003165,8.717713,0.000060
3,1,cosine_userknn,0.004701,0.025753,0.007918,0.004492,7.240762,0.000041
4,1,tfidf_userknn,0.007227,0.039631,0.012525,0.007582,7.359619,0.000044
5,1,bm25_userknn,0.002950,0.014813,0.005200,0.002739,8.948433,0.000073
6,2,cosine_userknn,0.004428,0.022962,0.007728,0.004140,7.379909,0.000047
7,2,tfidf_userknn,0.006805,0.036173,0.012028,0.007015,7.493861,0.000049
8,2,bm25_userknn,0.003132,0.014986,0.005544,0.002776,9.103093,0.000080
9,3,cosine_userknn,0.004443,0.022372,0.007777,0.004058,7.414437,0.000053


Так как ноутбук работал долго, а мы сохранили модели. То при запуске нового ноутбука просто подгрузим наши обученныйе модельки

А теперь превратим модели Популяпное и Юзеркнн в класс,чтобы показать, что мы обрабатываем холодных юзеров

In [127]:
interactions_df

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76.0,0.0
5476247,546862,9673,2021-04-13,2308.0,49.0
5476248,697262,15297,2021-08-20,18307.0,63.0
5476249,384202,16197,2021-04-19,6203.0,100.0


Я переписала код, как у меня в сервисе потому что класс, который учился, очень медленный. Но я использовала тот же dill, поэтому результат честный 

In [128]:
class UserKnn:
    """Class for fit-perdict UserKNN model
    """

    def __init__(self, model_path: str, nn: int = 50):
        with open(model_path, 'rb') as fh:
            self.model = dill.load(fh)
        self.dataset = interactions_df
        self.idf, self.idf_dict = self.prepare_idf()

        train_ids = np.load("/kaggle/input/111111/train_ids.npy",
                            allow_pickle=True)
        train = self.dataset.loc[train_ids]
        self.dataset = train

        self.watched, self.watched_dict = self.prepare_watched()

        self.get_mappings(train)

        self.mapper = self._generate_recs_mapper(N=nn)

    def recommend(self, user_id: int):
        pred_df = pd.DataFrame({
            'user_id': [user_id]
        })
        
        similar_users, similarity = zip(
            *pred_df['user_id'].map(self.mapper))

        pred_fast = []
        watched_items = {}
        for similar_user, similar in zip(similar_users[0], similarity[0]):
            if similar >= 1:
                continue
            for item_id in self.watched_dict[similar_user]:
                if item_id in watched_items:
                    continue
                watched_items[item_id] = None
                rank_idf = similar * self.idf_dict[item_id]

                new = [user_id, similar_user, similar, item_id, rank_idf]
                pred_fast.append(new)

        pred_fast.sort(key=lambda x: x[-1], reverse=True)
        predictions = [x[-2] for x in pred_fast][0:10]

        return predictions

    def get_mappings(self, train):
        self.users_inv_mapping = dict(enumerate(train['user_id'].unique()))
        self.users_mapping = {v: k for k, v in self.users_inv_mapping.items()}

        self.items_inv_mapping = dict(enumerate(train['item_id'].unique()))
        self.items_mapping = {v: k for k, v in self.items_inv_mapping.items()}

    def _generate_recs_mapper(self, N: int):
        def _recs_mapper(user):
            user_id = self.users_mapping[user]
            recs = self.model.similar_items(user_id, N=N)
            maps = [self.users_inv_mapping[user] for user in recs[0]], \
                list(sim for sim in recs[1])
            maps[0].pop(0)
            maps[1].pop(0)
            return maps

        return _recs_mapper

    def prepare_watched(self):
        watched = self.dataset.groupby('user_id').agg({'item_id': list})

        watched_dict = {user_id: items["item_id"] for user_id, items
                        in watched.iterrows()}
        return watched, watched_dict

    def get_watched_dict(self):
        return self.watched_dict

    def prepare_idf(self):
        cnt = Counter(self.dataset['item_id'].values)
        idf = pd.DataFrame.from_dict(cnt, orient='index',
                                     columns=['doc_freq']).reset_index()
        # num of documents = num of recommendation list = dataframe shape
        n = self.dataset.shape[0]
        # pylint: disable = E1137, E1136
        idf['idf'] = idf['doc_freq'].apply(lambda x:
                                           np.log((1 + n) / (1 + x) + 1))

        idf_dict = {items["index"]: items["idf"] for ind, items in
                    idf.iterrows()}
        return idf, idf_dict


In [129]:
user_knn = UserKnn("/kaggle/working/tfidf_userknn.dill")

Выдает все хорошо там, где есть юзеры

In [154]:
reco_knn = user_knn.recommend(1234)
reco_knn

[12581, 6658, 11214, 5477, 2491, 6706, 10853, 2072, 14551, 13436]

In [157]:
reco_knn = user_knn.recommend(3)
reco_knn

[4342, 16013, 16090, 10249, 15745, 9134, 6214, 15634, 7651, 4160]

Где юзеры холодные, то падает

In [132]:
reco_knn = user_knn.recommend(1235)
reco_knn

KeyError: 1235

In [144]:
items_df=items

In [149]:
class Popular:
    def __init__(self, model_path):
        super().__init__()
        self.mp_items = 1500
        with open(model_path, 'rb') as fh:
            self.pop = dill.load(fh)

        interactions=interactions_df
        items = items_df
        self.dataset = self.make_dataset_year_genre(interactions, items)

        self.popular_items = list(self.pop.recommend(
            [0],
            dataset=self.dataset,
            k=self.mp_items,
            filter_viewed=False)["item_id"])

    def recommend(self, _, k: int = 10):
        reco_list = self.popular_items[:k]

        return reco_list

    def get_popular_items(self):
        return self.popular_items

    @staticmethod
    def make_dataset_year_genre(interactions: pd.DataFrame,
                                items: pd.DataFrame):
        _, bins = pd.qcut(items["release_year"], 10, retbins=True)

        items_for_years = pd.DataFrame(
            {
                "id": items["item_id"],
                "value": pd.cut(items["release_year"], bins=bins,
                                labels=bins[:-1]),
                "feature": "release_year",
            }
        )

        items["genre"] = items["genres"].str.split(",")

        items_for_genre = items[["item_id", "genre"]].explode("genre")
        items_for_genre.columns = ["id", "value"]
        items_for_genre["feature"] = "genre"

        items_year_genre = pd.concat([items_for_genre, items_for_years])
        items_year_genre = items_year_genre[items_year_genre['id'].isin(
            interactions['item_id'])]

        dataset = Dataset.construct(
            interactions_df=interactions,
            user_features_df=None,
            item_features_df=items_year_genre,
            cat_item_features=['genre', 'release_year']
        )

        return dataset

In [145]:
class PopularUserKnn:
    def __init__(self, user_knn_model: UserKnn, pop_model: Popular, n=10):
        super().__init__()
        self.user_knn_model = user_knn_model
        self.popular_model = pop_model
        self.n = n
    def recommend(self, user_id: int): 
        try:
            reco_list = self.user_knn_model.recommend(user_id)
        except KeyError:
            # cold users fix
            reco_list = self.popular_model.get_popular_items()[:self.n]
            return reco_list

        # get up to N recos
        if len(reco_list) < self.n:
            new_reco = [*reco_list]
            watched = self.user_knn_model.get_watched_dict()[user_id]
            additional_reco = self.popular_model.get_popular_items()

            for rec in additional_reco:
                if len(new_reco) == self.n:
                    break
                if rec not in new_reco and rec not in watched:
                    new_reco.append(rec)
            if len(new_reco) < self.n:
                for rec in additional_reco:
                    if len(new_reco) == self.n:
                        break
                    if rec not in new_reco:
                        new_reco.append(rec)

            reco_list = new_reco
        if len(reco_list) != self.n:
            raise NotEnoughError(len(reco_list))

        return reco_list

In [146]:
model_pop_knn = PopularUserKnn(user_knn, Popular('/kaggle/working/models/popular.dill'))

Теперь все работает!

In [170]:
reco_knn = model_pop_knn.recommend(1)
reco_knn

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [172]:
reco_knn = model_pop_knn.recommend(3)
reco_knn

[4342, 16013, 16090, 10249, 15745, 9134, 6214, 15634, 7651, 4160]